In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
#from train import create_dir, load_dataset

In [2]:
global image_h
global image_w
global num_landmarks

In [3]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
# create_dir("results")

""" Hyperparameters """
image_h = 512
image_w = 512
num_landmarks = 106

""" Paths """
path = "./model"
model_path = os.path.join(path, "model_mobile.h5")

In [4]:
# def plot_landmarks(image, landmarks):
#     h, w, _ = image.shape
#     radius = int(h * 0.005)

#     for i in range(0, len(landmarks), 2):
#         x = int(landmarks[i] * w)
#         y = int(landmarks[i+1] * h)

#         image = cv2.circle(image, (x, y), radius, (255, 0, 0), -1)

#     return image

In [5]:
def plot_landmarks2(image, landmarks):
    h, w, _ = image.shape
    
    for i in range(0, len(landmarks), 2):
        x = int(landmarks[i]*w)
        y = int(landmarks[i+1]*h)
        #print(x,y)
        image = cv2.circle(image,(x, y), 2, (0,255,0), -1)
        num = int(i/2 + 1)
        num_text = str(num)
        text_origin = (x,y+10)
        TEXT_FACE = cv2.FONT_HERSHEY_DUPLEX
        TEXT_SCALE = 0.2
        TEXT_THICKNESS = 1
        image = cv2.putText(image, num_text, text_origin, TEXT_FACE, TEXT_SCALE, (0,0,255), TEXT_THICKNESS, cv2.LINE_AA)
    
    return image

In [7]:
""" Load the model """
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 256, 256, 16  432         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 256, 256, 16  64          ['Conv1[0][0]']                  
                                )                                                             

                                                                                                  
 block_3_expand (Conv2D)        (None, 128, 128, 96  1536        ['block_2_add[0][0]']            
                                )                                                                 
                                                                                                  
 block_3_expand_BN (BatchNormal  (None, 128, 128, 96  384        ['block_3_expand[0][0]']         
 ization)                       )                                                                 
                                                                                                  
 block_3_expand_relu (ReLU)     (None, 128, 128, 96  0           ['block_3_expand_BN[0][0]']      
                                )                                                                 
                                                                                                  
 block_3_p

                                                                                                  
 block_6_depthwise_BN (BatchNor  (None, 32, 32, 96)  384         ['block_6_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_6_depthwise_relu (ReLU)  (None, 32, 32, 96)   0           ['block_6_depthwise_BN[0][0]']   
                                                                                                  
 block_6_project (Conv2D)       (None, 32, 32, 32)   3072        ['block_6_depthwise_relu[0][0]'] 
                                                                                                  
 block_6_project_BN (BatchNorma  (None, 32, 32, 32)  128         ['block_6_project[0][0]']        
 lization)                                                                                        
          

 lization)                                                                                        
                                                                                                  
 block_10_expand_relu (ReLU)    (None, 32, 32, 192)  0           ['block_10_expand_BN[0][0]']     
                                                                                                  
 block_10_depthwise (DepthwiseC  (None, 32, 32, 192)  1728       ['block_10_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_10_depthwise_BN (BatchNo  (None, 32, 32, 192)  768        ['block_10_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_10_

 alization)                                                                                       
                                                                                                  
 block_14_expand (Conv2D)       (None, 16, 16, 480)  38400       ['block_13_project_BN[0][0]']    
                                                                                                  
 block_14_expand_BN (BatchNorma  (None, 16, 16, 480)  1920       ['block_14_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_14_expand_relu (ReLU)    (None, 16, 16, 480)  0           ['block_14_expand_BN[0][0]']     
                                                                                                  
 block_14_depthwise (DepthwiseC  (None, 16, 16, 480)  4320       ['block_14_expand_relu[0][0]']   
 onv2D)   

 dense (Dense)                  (None, 212)          271572      ['dropout[0][0]']                
                                                                                                  
Total params: 977,796
Trainable params: 959,252
Non-trainable params: 18,544
__________________________________________________________________________________________________


In [8]:
x = './Brad_Pitt.png'
x

'./Brad_Pitt.png'

In [9]:
pwd

'C:\\Users\\Admin'

In [10]:
""" Reading the image """
image = cv2.imread(x, cv2.IMREAD_COLOR)
image_x = image

image = cv2.resize(image, (image_w, image_h))

image = image/255.0 ## (512, 512, 3)
image = np.expand_dims(image, axis=0) ## (1, 512, 512, 3)
image = image.astype(np.float32)
image

array([[[[0.2509804 , 0.32156864, 0.34117648],
         [0.24705882, 0.32156864, 0.34117648],
         [0.24705882, 0.32156864, 0.34117648],
         ...,
         [0.23921569, 0.36078432, 0.37254903],
         [0.23921569, 0.36078432, 0.37254903],
         [0.23921569, 0.36078432, 0.37254903]],

        [[0.24705882, 0.32156864, 0.34117648],
         [0.2509804 , 0.32156864, 0.34117648],
         [0.2509804 , 0.3254902 , 0.34117648],
         ...,
         [0.23921569, 0.35686275, 0.37254903],
         [0.23921569, 0.35686275, 0.37254903],
         [0.23921569, 0.35686275, 0.37254903]],

        [[0.2509804 , 0.32156864, 0.34509805],
         [0.2509804 , 0.3254902 , 0.34509805],
         [0.25490198, 0.3254902 , 0.34509805],
         ...,
         [0.24313726, 0.35686275, 0.37254903],
         [0.24313726, 0.35686275, 0.37254903],
         [0.24313726, 0.35686275, 0.37254903]],

        ...,

        [[0.16470589, 0.1254902 , 0.10980392],
         [0.16470589, 0.12156863, 0.10588235]

In [11]:
""" Prediction """
pred = model.predict(image, verbose=0)[0]
pred = pred.astype(np.float32)
pred

array([0.295754  , 0.34953576, 0.29536876, 0.37233624, 0.2939488 ,
       0.39541063, 0.29285085, 0.41659755, 0.29559624, 0.43934116,
       0.2974852 , 0.4600561 , 0.3030072 , 0.48247913, 0.30756575,
       0.5061577 , 0.3130786 , 0.528417  , 0.32436132, 0.54882187,
       0.33650964, 0.5691425 , 0.35426518, 0.5852487 , 0.37182865,
       0.60081804, 0.39293784, 0.61439466, 0.4145758 , 0.6214441 ,
       0.44033214, 0.6248505 , 0.46825892, 0.6260994 , 0.4954552 ,
       0.6228098 , 0.522391  , 0.6201984 , 0.54687893, 0.6131589 ,
       0.57148564, 0.59996915, 0.59324855, 0.5884598 , 0.6102371 ,
       0.5714292 , 0.6249613 , 0.5539015 , 0.63657844, 0.53203714,
       0.6465676 , 0.50773525, 0.6527968 , 0.48418984, 0.6576056 ,
       0.45755792, 0.65946704, 0.43667215, 0.6617074 , 0.41257346,
       0.66355073, 0.39063698, 0.6601421 , 0.3648152 , 0.6573598 ,
       0.33967903, 0.32984275, 0.3371966 , 0.35017216, 0.31827185,
       0.37584046, 0.31149727, 0.4043654 , 0.31242692, 0.43117

In [12]:
""" Saving the results """
#gt_landmarks = plot_landmarks(image_x.copy(), landmarks)
result_image = plot_landmarks2(image_x.copy(), pred)
# line = np.ones((image_x.shape[0], 10, 3)) * 255

In [13]:
print(type(result_image))

<class 'numpy.ndarray'>


In [14]:
result_image

array([[[64, 82, 87],
        [63, 82, 87],
        [63, 81, 86],
        ...,
        [62, 91, 96],
        [61, 92, 95],
        [61, 92, 95]],

       [[63, 82, 87],
        [65, 83, 88],
        [64, 82, 87],
        ...,
        [62, 91, 96],
        [61, 91, 95],
        [61, 91, 95]],

       [[64, 82, 88],
        [65, 84, 89],
        [65, 83, 88],
        ...,
        [63, 91, 96],
        [62, 91, 95],
        [62, 91, 95]],

       ...,

       [[42, 32, 28],
        [42, 31, 27],
        [41, 30, 26],
        ...,
        [44, 36, 34],
        [42, 32, 30],
        [50, 41, 38]],

       [[44, 34, 29],
        [42, 31, 27],
        [41, 30, 26],
        ...,
        [41, 34, 31],
        [43, 34, 32],
        [44, 35, 32]],

       [[43, 32, 28],
        [40, 30, 25],
        [40, 30, 26],
        ...,
        [42, 35, 32],
        [40, 31, 29],
        [42, 33, 30]]], dtype=uint8)

In [15]:
# cat_image = np.concatenate([gt_landmarks, line, pred_landmarks], axis=1)
name = "brad_mobile_epoch40"
cv2.imwrite(f"./test_result/{name}.png", result_image)

False

In [16]:
# Step1: just put sticker on the exact points
# Step2: adjust the position of sticker
# Step3: scale size of sticker

In [17]:
def put_sticker(image, landmarks, sticker):
    h, w, _ = image.shape
    
    larger_image = image
    smaller_image = sticker
    
    position_x = 100  # X-coordinate of the top-left corner of the smaller image
    position_y = 50   # Y-coordinate of the top-left corner of the smaller image
    smaller_height, smaller_width, _ = smaller_image.shape
    
    end_x = position_x + smaller_width
    end_y = position_y + smaller_height
    
    larger_image[position_y:end_y, position_x:end_x] = smaller_image
    return larger_image

In [18]:
pwd

'C:\\Users\\Admin'

In [19]:
larger_image = cv2.imread('Brad_Pitt.png')
smaller_image = cv2.imread('./Stickers/bat.png')
larger_height, larger_width, _ = larger_image.shape

desired_scale = 0.4  # Desired scale for the smaller image (e.g., 0.5 for half the size)


smaller_height = int(larger_height * desired_scale)
smaller_width = int(larger_width * desired_scale)
rescaled_smaller = cv2.resize(smaller_image, (smaller_width, smaller_height))


position_x = pred[0]*255   # X-coordinate of the top-left corner of the smaller image
position_y = pred[1]*255   # Y-coordinate of the top-left corner of the smaller image
smaller_height, smaller_width, _ = rescaled_smaller.shape
    
end_x = position_x + smaller_width
end_y = position_y + smaller_height

print(larger_height, larger_width)
print(position_x, position_y)
print(smaller_height, smaller_width)
print(end_x, end_y)

NameError: name 'desired_scale' is not defined

In [35]:
# overlay = larger_image.copy()
# overlay[int(position_y):int(end_y), int(position_x):int(end_x)] = rescaled_smaller

# result = cv2.add(larger_image, overlay)

#https://stackoverflow.com/questions/14063070/overlay-a-smaller-image-on-a-larger-image-python-opencv

s_img = cv2.imread('./bat.png')
l_img = cv2.imread('Brad_Pitt.png')
# x_offset=y_offset=50    # x_offset, y_offset = position_x, position_y

larger_height, larger_width, _ = l_img.shape
desired_scale = 0.4  # Desired scale for the smaller image (e.g., 0.5 for half the size)




smaller_height = int(larger_height * desired_scale)
smaller_width = int(larger_width * desired_scale)
rescaled_smaller = cv2.resize(s_img, (smaller_width, smaller_height))

smaller_height, smaller_width, _ = rescaled_smaller.shape

x_offset = pred[0]*255  # X-coordinate of the top-left corner of the smaller image
y_offset = pred[1]*255 

print(larger_height, larger_width)
print(smaller_height, smaller_width)
print(x_offset, y_offset)

467 378
186 151
75.41726633906364 89.13161963224411


In [34]:
l_img[int(y_offset):int(y_offset+smaller_height), int(x_offset):int(x_offset+smaller_width)] = s_img

y1, y2 = y_offset, y_offset + s_img.shape[0]
x1, x2 = x_offset, x_offset + s_img.shape[1]

alpha_s = s_img[:, :, 3] / 255.0
alpha_l = 1.0 - alpha_s

for c in range(0, 3):
    l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] +
                              alpha_l * l_img[y1:y2, x1:x2, c])

ValueError: could not broadcast input array from shape (423,360,3) into shape (186,151,3)

In [ ]:
name = "output_sticker_on"
cv2.imwrite(f"./test_result/{name}.png", result)

In [ ]:
cv2.imshow('Result', larger_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Alternatively, to save the resulting image:
cv2.imwrite('path_to_output_image.jpg', larger_image)